In [475]:
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'

In [476]:
#import 4 files - using seperate columns and data from each
ffdf = pd.read_csv('FFB_top200.csv') #fantasy footballers rankings
fptdf = pd.read_csv('FPS_TOTS.csv') #fantasy pros total points rankings
fpadf = pd.read_csv('FPS_AVGS.csv') #fantasy pros avergage points rankings
fpodf = pd.read_csv('FPS_OVERVIEW.csv') #fantasy pros overall rankings

In [477]:
ffdf.head()

,Name,Team,Pos,Position,Rank,Andy,Jason,Mike,Markers
0,Christian McCaffrey,CAR,RB,RB,1,1,2,2,Mark Drafted Mark Keeper
1,Jonathan Taylor,IND,RB,RB,2,2,1,3,Mark Drafted Mark Keeper
2,Cooper Kupp,LAR,WR,WR,3,5,3,4,Mark Drafted Mark Keeper
3,Justin Jefferson,MIN,WR,WR,4,6,4,1,Mark Drafted Mark Keeper
4,Austin Ekeler,LAC,RB,RB,5,4,6,5,Mark Drafted Mark Keeper


In [478]:
#matching team names for both sites to merge properly
filt = (ffdf['Team'] == 'JAX')
ffdf[filt]
ffdf.loc[filt, 'Team'] = 'JAC'

In [479]:
#match ffb column name to FP data
ffdf.rename(columns={'Name': 'PLAYER', 'Rank': 'FFB RK'},inplace=True)

In [480]:
#capitalize all col headings
ffdf.columns = [x.upper() for x in ffdf.columns]

In [481]:
#remove unwanted cols
ffdf.drop(columns=['JASON', 'MIKE', 'MARKERS', 'POSITION'], inplace=True)

In [482]:
fpodf.drop(columns=['RK','TIERS','TEAM','SOS SEASON'], inplace=True)

In [483]:
fpodf.rename(columns={'PLAYER NAME': 'PLAYER', 'BYE WEEK': 'BYE',
                     'ECR VS. ADP': 'ECR V ADP'},inplace=True)

In [484]:
fpadf.drop(columns=['RK','TIERS','TEAM','YDS','TDS','REC','YDS.1',
                    'TDS.1','ATT','YDS.2','TDS.2'], inplace=True)

In [485]:
fpadf.rename(columns={'PLAYER NAME': 'PLAYER', 'FAN PTS': 'PPG',},inplace=True)

In [486]:
#function to change POS to just POS not rank in order to merge dfs more accurately
def change_pos(posrnk):
    pos = posrnk[0:2]
    return pos

In [487]:
#apply above function
fpodf['POS'] = fpodf['POS'].apply(change_pos)

In [488]:
fpodf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 486 entries, 0 to 485
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   PLAYER     486 non-null    object
 1   POS        486 non-null    object
 2   BYE        486 non-null    object
 3   ECR V ADP  486 non-null    object
dtypes: object(4)
memory usage: 15.3+ KB


In [489]:
ecr = 0

In [490]:
#function to generate adp col
def gen_adp(eva):
    global ecr
    ecr +=1
    if eva == '-':
        return '-'
    return int(eva) + ecr
    

In [491]:
#run fxn on ecr v adp col and generate new adp col
fpodf['ADP'] = fpodf['ECR V ADP'].apply(gen_adp)

In [492]:
fpodf.drop(columns=['ECR V ADP'], inplace=True)

In [493]:
fptdf.rename(columns={'RK': 'FPS RK',
                      'TIERS': 'TIER',
                      'PLAYER NAME': 'PLAYER',
                      'FAN PTS': 'TOT PTS',
                      'YDS': 'PYD',
                      'TDS': 'PTD',
                      'YDS.1': 'REC YD',
                      'TDS.1': 'REC TD',
                      'ATT': 'RUS ATT',
                      'YDS.2': 'RUS YD',
                      'TDS.2': 'RUS TD'}, inplace=True)

In [494]:
#combining the 2 fantasy pros data frames (including avg points onto the total points df)
fpdf = pd.merge(fptdf, fpadf, on='PLAYER')

In [495]:
#merging with the Overall FP rankings to make 1 df containing all relevant FP data (from 3 dfs)
fpdf = pd.merge(fpdf, fpodf, on='PLAYER')

In [496]:
#function to change all player names into an equal value
#discards JR, III, ', ., etc from names and capitalizes them

def update_name(player):
    punct = ['.',',',"'",'-']
    letter_list = [let.upper() for let in player if let not in punct]
    player_str = ''.join(letter_list)
    player_list = player_str.split()
    if len(player_list[0]) > 2:
        player_list[0] = player_list[0][0:3] 
    #makes first name only 3 letters (avoids abbreviated names like kenneth->ken, patrick->pat)
        #if the players name is more than 2 letters (AJ)
    if len(player_list) == 3:
        if len(player_list[-1]) < 4:
            player_list = player_list[0:-1]
        return ' '.join(player_list)
    else:
        return str(player_list[0]) + ' ' + str(player_list[1])

In [497]:
#run function on all player names for both dfs
ffdf['PLAYER'] = ffdf['PLAYER'].apply(update_name)

In [498]:
fpdf['PLAYER'] = fpdf['PLAYER'].apply(update_name)

In [499]:
#making a new col to include player, team, and pos. this will avoid errors with players with the same name
#ex Jamall Williams and Jameson williams, both play for DET but different positions
#need to do this before we merge both sites data
fpdf['PLAYER POS TM'] = fpdf['PLAYER'] + ', ' + fpdf['POS'] + ', ' + fpdf['TEAM']
ffdf['PLAYER POS TM'] = ffdf['PLAYER'] + ', ' + ffdf['POS'] + ', ' + ffdf['TEAM']

In [500]:
#removing PLAYER, POS, TEAM from one df so there arent duplicate messy cols on the merge
ffdf.drop(columns=['PLAYER', 'POS', 'TEAM'], inplace=True)

In [501]:
#combine FPS and FFB data frames 
combined_df = pd.merge(fpdf, ffdf, on='PLAYER POS TM')
#this will merge a player with the same 
#first 3 or 2 letters of first name, team, position
#includes only players on both rankings

combined_df.shape

In [502]:
#combined_full_df = pd.merge(fpdf, ffdf, how='outer', on='PLAYER')
#includes all players, also makes calculations NaN values which makes data messy 

In [503]:
#shows the entire df
pd.set_option('display.max_rows', None)

In [504]:
#make a new column with the average rank of the 2 sites 
combined_df['AVG RK'] = (combined_df['FPS RK'] + combined_df['FFB RK'])/2

In [505]:
#make index the average rank of the 2 sites 
#combined_df.set_index('AVG RK', inplace=True)

In [506]:
#IF AVG rank is tied it sorts secondarily by andys rank
combined_df.sort_values(by=['AVG RK', 'ANDY'], inplace=True)

In [507]:
#combined_df.info()
#changing relevant numbers to ints (clean up space) if numbers are float/null
#combined_df['TIER'] = combined_df['TIER'].apply(lambda x: int(x) if type(x)=='float' else x)
#lambda x: int(x) if isinstance(x, float) else x
#combined_df[['TIER','FFB RK']].fillna('').astype(int)
#combined_df['TIER'] = combined_df['TIER'].apply(lambda x: int(x) if x>=1 else x

In [508]:
#initial values set for counters to make new positional ranks cols
qbs = 0
rbs = 0
wrs = 0
tes = 0

In [509]:
#function used to generate a new column with PROJ POS, meaning the players rank for their pos
#QB1, RB7 for ex
def projected_pos(pos):
    
    #counters set above
    global qbs 
    global rbs
    global wrs
    global tes
    
    if str(pos) == 'QB':
        qbs +=1
        return pos + ' ' + str(qbs)
    elif str(pos) == 'RB':
        rbs +=1
        return pos + ' ' + str(rbs)
    elif str(pos) == 'WR':
        wrs +=1
        return pos + ' ' + str(wrs)
    elif str(pos) == 'TE':
        tes +=1
        return pos + ' ' + str(tes)

In [510]:
#run fxn on pos col and generate new column
combined_df['PROJ'] = combined_df['POS'].apply(projected_pos)

In [511]:
#initial values set for counters to make new positional ranks cols for last years finishing ranks
qbs2 = 0
rbs2 = 0
wrs2 = 0
tes2 = 0
flex = 0

In [512]:
#function used to generate a new column with 2021 POS, meaning the players rank for their pos
#at the end of the previous season
#if the player did not play last year (rookie) the value will be NA
#also includes in () if the player is WR RB or TE the overall flex rank, not just positional
#QB1, RB7 for ex
def last_season_pos(pos, total_points):
    
    #counters set above
    global qbs2
    global rbs2
    global wrs2
    global tes2
    global flex
    
    if total_points == 0.0:
        return 'NA'
    elif str(pos) == 'QB':
        qbs2 +=1
        return pos + ' ' + str(qbs2)
    elif str(pos) == 'RB':
        flex +=1
        rbs2 +=1
        return pos + ' ' + str(rbs2) + ' ' + '(' + str(flex) + ')'
    elif str(pos) == 'WR':
        flex +=1
        wrs2 +=1
        return pos + ' ' + str(wrs2) + ' ' + '(' + str(flex) + ')'
    elif str(pos) == 'TE':
        flex +=1
        tes2 +=1
        return pos + ' ' + str(tes2) + ' ' + '(' + str(flex) + ')'

In [513]:
#need to sort values by last years points before we run function, will change back after 
combined_df.sort_values(by=['TOT PTS'], ascending=False, inplace=True)

In [514]:
#executes function using the POS and TOT PTS to create new col 2021 RK (where the 
#player finished last season)
combined_df['FINISH'] = combined_df.apply(lambda x: last_season_pos(x['POS'], x['TOT PTS']), axis=1)

In [515]:
#change sort back to the AVG RK
combined_df.sort_values(by=['AVG RK', 'ANDY'], inplace=True)

In [516]:
#can now drop POS, player, team columns
combined_df.drop(columns=['POS', 'PLAYER', 'TEAM'], inplace=True)
combined_df.rename(columns={'PLAYER POS TM': 'PLAYER'},inplace=True)
#combined_df.head()

In [517]:
#now that merge is completed with accuracy of player name, team, and pos, we can seperate out pos and team
combined_df[['PLAYER','POS','TEAM']] = combined_df['PLAYER'].str.split(', ', expand=True)

In [518]:
#pos_grp = combined_df.groupby(['POS'])
#pos_grp.get_group('RB')

In [519]:
#function to if applicable subtract the players finishing position last year minus their draft position this year
    #positive values mean they finished positionally better than they are being drafted (good)
    #negative values mean they are being drafted higher than they finished positionally last year (bad)
def last_vs_proj(finish, proj):
    if finish == 'NA':
        return 0
    else:
        finish_list = finish.split()
        proj_list = proj.split()
        value = int(proj_list[1]) - int(finish_list[1])
        return value 

In [520]:
combined_df['VALUE'] = combined_df.apply(lambda x: last_vs_proj(x['FINISH'], x['PROJ']), axis=1)

In [521]:
#function to find difference between FFB and FPS rankings 
#both ranks are in the DF as ints already
def correlation(ffbs, fpros):
    return abs(ffbs-fpros)
    

In [522]:
rank_counter = 0

In [523]:
#making a new column for numbering the picks instead of indexing by avg rk
def total_rank(x):
    global rank_counter
    rank_counter +=1
    return rank_counter

In [524]:
combined_df['RK'] = combined_df['PLAYER'].apply(total_rank)

In [525]:
#making new RANKS VS ADP COL INSTEAD OF FANTASY PROS ECR VS ADP
def rkvadp(rank, adp):
    
    if adp == '-':
        return '-'
    
    return int(adp) - int(rank)


In [526]:
combined_df['RK V ADP'] = combined_df.apply(lambda x: rkvadp(x["RK"], x["ADP"]), axis=1)

In [527]:
combined_df.set_index('RK', inplace=True)

In [528]:
combined_df['DIFF'] = combined_df.apply(lambda x: correlation(x["FPS RK"], x["FFB RK"]), axis=1)

In [529]:
#organizing columns
combined_df = combined_df.reindex(columns=['TIER',
                                           'PLAYER',
                                           'TEAM',
                                           'PROJ',
                                           'AVG RK',
                                           'FFB RK',
                                           'FPS RK',
                                           'ANDY',
                                           'DIFF',
                                           'TOT PTS',
                                           'PPG',
                                           'FINISH',
                                           'VALUE',
                                           'REC',
                                           'REC YD',
                                           'REC TD',
                                           'RUS ATT',
                                           'RUS YD',
                                           'RUS TD',
                                           'PYD',
                                           'PTD',
                                           'POS',
                                           'RK V ADP',
                                           'ADP',
                                           'BYE',])

In [530]:
#combined_df.sort_values(by=['VALUE'], ascending=False)
#can do this positionally as well, make new DF for each position

In [531]:
#combined_df.sort_values(by=['DIFF'], ascending=False)

In [532]:
#converting to csv
#should make DFs sorted by total, positional (print D too), value

#Can use this and eliminate the POS column
#qbs = combined_df['PROJ'].str.contains('QB')
#type(combined_df[qbs]) #dataframe, can export on this

#combined_df.to_csv('07_19_22_CombinedRanks.csv')

In [533]:
#MAKING POSITIONAL DATAFRAMES 
#if eliminating the POS column can do:
#qbs = combined_df['PROJ'].str.contains('QB')
#type(combined_df[qbs]) #dataframe
'''
pos_grp = combined_df.groupby(['POS'])
qb_df = pos_grp.get_group('QB')
rb_df = pos_grp.get_group('RB')
wr_df = pos_grp.get_group('WR')
te_df = pos_grp.get_group('TE')
'''

"\npos_grp = combined_df.groupby(['POS'])\nqb_df = pos_grp.get_group('QB')\nrb_df = pos_grp.get_group('RB')\nwr_df = pos_grp.get_group('WR')\nte_df = pos_grp.get_group('TE')\n"

In [534]:
pd.set_option('display.max_columns', None)

In [535]:
#CUSTOMIZING EACH POSITIONAL DF

In [536]:
'''
qb_df.drop(columns=['REC', 'REC YD', 'REC TD', 'POS'], inplace=True)
rb_df.drop(columns=['PYD', 'PTD', 'POS'], inplace=True)
wr_df.drop(columns=['PYD', 'PTD', 'POS'], inplace=True)
te_df.drop(columns=['RUS ATT','RUS YD','RUS TD','PYD', 'PTD', 'POS'], inplace=True)
'''

"\nqb_df.drop(columns=['REC', 'REC YD', 'REC TD', 'POS'], inplace=True)\nrb_df.drop(columns=['PYD', 'PTD', 'POS'], inplace=True)\nwr_df.drop(columns=['PYD', 'PTD', 'POS'], inplace=True)\nte_df.drop(columns=['RUS ATT','RUS YD','RUS TD','PYD', 'PTD', 'POS'], inplace=True)\n"

In [537]:
combined_df.head()

,TIER,PLAYER,TEAM,PROJ,AVG RK,FFB RK,FPS RK,ANDY,DIFF,TOT PTS,PPG,FINISH,VALUE,REC,REC YD,REC TD,RUS ATT,RUS YD,RUS TD,PYD,PTD,POS,RK V ADP,ADP,BYE
RK,,,,,,,,,,,,,,,,,,,,,,,,,
1,1,CHR MCCAFFREY,CAR,RB 1,1.5,1,2,1,1,109.0,15.6,RB 35 (97),-34,37,343,1,99,442,1,0,0,RB,2,3,13
2,1,JON TAYLOR,IND,RB 2,1.5,2,1,2,1,353.1,20.8,RB 1 (2),1,40,360,2,332,1811,18,0,0,RB,-1,1,14
3,1,AUS EKELER,LAC,RB 3,4.0,5,3,4,2,308.8,19.3,RB 2 (3),1,70,647,8,206,911,12,0,0,RB,-1,2,8
4,1,JUS JEFFERSON,MIN,WR 1,4.0,4,4,6,0,276.4,16.3,WR 4 (6),-3,108,1616,10,6,14,0,0,0,WR,2,6,7
5,2,COO KUPP,LAR,WR 2,4.5,3,6,5,3,367.0,21.6,WR 1 (1),1,145,1947,16,4,18,0,0,0,WR,-1,4,7


In [538]:
#flex_filt = (combined_df['POS'] != 'QB')
#ppg_df = combined_df[flex_filt].sort_values(by='PPG', ascending=False).head(30)
#ppg_df.to_csv('2021 Flex PPG top 30')